## Actividad: Análisis de datos con Spark.
### 1. Importación de librerías y carga de datos
En primer lugar, importamos las librerías que utilizaremos para la actividad y, seguidamente, configuramos e inicializamos Spark. Lo configuramos para funcionar en modo local y le ponemos un nombre a nuestra aplicación, en este caso, "contratos_app".

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import sum

spark_conf = SparkConf().setMaster("local").setAppName("contratos_app")
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

Cargamos los dos ficheros csv como dataframes con inferencia de esquema y mostramos por pantalla los primeros cinco registros y el esquema de cada dataframe para comprobar que se han cargado bien los datos.

In [2]:
# Carga del csv de contratos por municipio.
contracts_df = spark.read.options(delimiter=";", header=True, inferSchema=True).csv("./Contratos_por_municipio.csv")
contracts_df.show(5)
contracts_df.printSchema()

# Carga del csv de comunidades y provincias.
communities_df = spark.read.options(delimiter=";", header=True, inferSchema=True).csv("./Comunidades_y_provincias.csv")
communities_df.show(5)
communities_df.printSchema()

+----------+---------+---------+---------------+-----------------+-----------------+
|codigo_mes|provincia|municipio|total_contratos|contratos_hombres|contratos_mujeres|
+----------+---------+---------+---------------+-----------------+-----------------+
|    201601|  Almeria|     Abla|             27|               20|                7|
|    201601|  Almeria| Abrucena|             39|               26|               13|
|    201601|  Almeria|     Adra|            456|              259|              197|
|    201601|  Almeria|Albanchez|              7|                4|                3|
|    201601|  Almeria|Alboloduy|              6|                3|                3|
+----------+---------+---------+---------------+-----------------+-----------------+
only showing top 5 rows

root
 |-- codigo_mes: integer (nullable = true)
 |-- provincia: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- total_contratos: integer (nullable = true)
 |-- contratos_hombres: integer 

### 2. Operaciones sobre los dataframes
Con el objetivo de contestar a la pregunta que se plantea en la actividad, **¿Qué comunidades autónomas han realizado más contratos a mujeres que a hombres durante todo el periodo?**, se realizarán las siguientes operaciones sobre los dataframes:
- Join.
- GroupBy.
- Agg.
- Filter.

Empezamos utilizando la función `join` para unir los registros de contratos y comunidades que tengan el mismo valor en la columna "provincia" y guardamos el resultado en un nuevo dataframe al que llamaremos "joined_df". Mostramos por pantalla los cinco primeros resultados del nuevo dataframe.

In [3]:
joined_df = contracts_df.join(communities_df, contracts_df.provincia == communities_df.Provincia, 'inner')

joined_df.show(5)

+----------+---------+---------+---------------+-----------------+-----------------+------------------+---------+
|codigo_mes|provincia|municipio|total_contratos|contratos_hombres|contratos_mujeres|Comunidad_Autonoma|Provincia|
+----------+---------+---------+---------------+-----------------+-----------------+------------------+---------+
|    201601|  Almeria|     Abla|             27|               20|                7|         Andalucia|  Almeria|
|    201601|  Almeria| Abrucena|             39|               26|               13|         Andalucia|  Almeria|
|    201601|  Almeria|     Adra|            456|              259|              197|         Andalucia|  Almeria|
|    201601|  Almeria|Albanchez|              7|                4|                3|         Andalucia|  Almeria|
|    201601|  Almeria|Alboloduy|              6|                3|                3|         Andalucia|  Almeria|
+----------+---------+---------+---------------+-----------------+-----------------+----

A continuación, utilizamos las funciones `groupBy` y `agg` para obtener la suma total de contratos de cada comunidad. Para ello, primero agrupamos por comunidad autónoma y como agregación calculamos la suma de los contratos. Además, renombramos las nuevas columnas haciendo uso del alias. Mostramos por pantalla los cinco primeros resultados de la nueva variable que contiene la agrupación.

In [4]:
contracts_by_community = joined_df \
    .groupBy(joined_df.Comunidad_Autonoma) \
    .agg(sum(joined_df.contratos_hombres).alias("num_contratos_hombres"), sum(joined_df.contratos_mujeres).alias("num_contratos_mujeres"))

contracts_by_community.show(5)

+--------------------+---------------------+---------------------+
|  Comunidad_Autonoma|num_contratos_hombres|num_contratos_mujeres|
+--------------------+---------------------+---------------------+
|Comunitat Valenciana|               608068|               436580|
|           Cantabria|                67918|                67813|
|   Murcia, Region de|               378124|               180291|
|         Extremadura|               213457|               136266|
|             Galicia|               278054|               250900|
+--------------------+---------------------+---------------------+
only showing top 5 rows



Finalmente, utilizamos la función `filter` para quedarnos únicamente con los registros que cumplan la condición de haber realizado más contratos a mujeres que a hombres; guardamos el resultado en una nueva variable a la que llamaremos "more_woman" y como siempre, mostramos por pantalla los cinco primeros resultados.

In [5]:
more_woman = contracts_by_community.filter(contracts_by_community.num_contratos_hombres < contracts_by_community.num_contratos_mujeres)

more_woman.show(5)

+--------------------+---------------------+---------------------+
|  Comunidad_Autonoma|num_contratos_hombres|num_contratos_mujeres|
+--------------------+---------------------+---------------------+
|Navarra, Comunida...|                97782|               104844|
|          Pais Vasco|               259770|               261310|
|             Melilla|                 5188|                 5708|
+--------------------+---------------------+---------------------+



### 3. Resultado
Para terminar, escribimos el resultado en csv utilizando `coalesce(1)` para que Spark escriba todo en un solo archivo.

In [6]:
more_woman.coalesce(1).write.mode('overwrite').option("header", "true").csv("./mas_contratos_a_mujeres")